In [38]:
# Install these packages before running the code

# !conda install -c conda-forge gdal -y
# !conda install rasterio
# !conda install boto3
# !conda install botocore

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.9.0
  latest version: 24.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.11.1



# All requested packages already installed.



In [39]:
#imports
import os
import numpy as np
import rasterio
from rasterio.mask import mask
from collections import defaultdict
import logging
import boto3
import botocore

ImportError: /opt/conda/envs/preprocessing/lib/python3.10/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /opt/conda/envs/preprocessing/lib/python3.10/site-packages/rasterio/../../../libgdal.so.36)

In [37]:
SCE_folder = "/"
input_folder_name = "input_dir"
output_folder_name = "output_dir"

current_dir = os.getcwd()
def create_folder(folder_name):
    folder_path = os.path.join(current_dir, folder_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")
    else:
        print(f"Folder '{folder_path}' already exists.")

create_folder(input_folder_name)
create_folder(output_folder_name)

/data-store/iplant/home/dakshasinghal/download_and_preprocessing
/data-store/iplant/home/dakshasinghal/download_and_preprocessing/input_dir
Folder '/data-store/iplant/home/dakshasinghal/download_and_preprocessing/input_dir' created.
/data-store/iplant/home/dakshasinghal/download_and_preprocessing/output_dir
Folder '/data-store/iplant/home/dakshasinghal/download_and_preprocessing/output_dir' created.


In [ ]:
# update the temporary credentials
def get_aws_creds():
 return {}

In [10]:

def upload_file(file_name, bucket, object_name=None):
    print("Upload", file_name, "to s3 bucket" )
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3',**get_aws_creds())
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except Exception as e:
        logging.error(e)
        return False
    return True


def delete_files(files):
  print(files)
  for file in files:
    if(os.path.isfile(file)):
      os.remove(file)


def download_from_s3(s3_client, bucket, file, download_dest):
  s3_client.download_file(bucket, file, download_dest)


def get_s3_files(s3, bucket, prefix):
  paginator = s3.get_paginator('list_objects_v2')
  pages = paginator.paginate(Bucket=bucket, Prefix=prefix)

  files = []
  for page in pages:
      for i in page['Contents']:
        if i['Key']!= prefix+"/":
          files.append(i['Key'].split("/")[1])
  return files


def grouping_files(files_list):
  groups = defaultdict(list)

  for file in files_list:
      key = "".join(file.split("_")[0:6])
      groups[key].append(file)

  grouped_strings = list(groups.values())

  return grouped_strings

def resample_bands(resampling_dir):
  print("Resampling Bands")
  bands_to_resample = ["B05", "B06", "B07","B11","B12"]
  files_list = os.listdir(resampling_dir)
  for file in files_list:
    band = file.split("_")[-1].split(".")[0]
    if(band in bands_to_resample and ".jp2" in file):
      file_path  = resampling_dir + file
      gdal.Warp(file_path, file_path, xRes=10, yRes=10)

def read_jp2(image_path):
    with rasterio.open(image_path) as src:
        image = src.read(1)  # Read the first band
        profile = src.profile
    return image, profile

In [ ]:
def apply_detfoo_mask(spectral_band, detfoo_mask):
    masked_band = np.where(detfoo_mask != 1, spectral_band, np.nan)  # Masking with NaN
    return masked_band

def mask_clouds_and_stack_bands(band_files, detfoo_files, output_file):
    print("Masking Clouds and Stacking Bands")

    masked_arrays = {}
    for band_file in band_files:
        directory, file_name = band_file.rsplit('/', 1)
        base_name, band = file_name.rsplit('_', 1)
        detfoo_file = "/content/drive/MyDrive/Earth Lab/Dry mapping/input_dir/"+f"{base_name}_qi_MSK_DETFOO_{band.replace('.jp2', '.gml')}"
        gdf = gpd.read_file(detfoo_file)
        with rasterio.open(band_file, driver = "JP2OpenJPEG") as src:
            band_crs = src.crs
            width = src.width
            height = src.height
            out_image, out_transform = mask(src, gdf.geometry, invert=True, all_touched=True, nodata=src.nodata, crop=False)
            print(gdf.geometry)
            band_num = int(band_file.split(".")[0][-2:])
            masked_arrays[band_num] = out_image
            print(masked_arrays[band_num])
    masked_image = rasterio.open(output_file, 'w', driver = "Gtiff", count=len(band_files), crs = band_crs, transform = out_transform, width = width, height = height, dtype = masked_arrays[2].dtype)
    index = 1
    for key in masked_arrays:
      masked_image.write(masked_arrays[key][0], indexes=index)
      index+=1

    masked_image.close()


if __name__ == "__main__":
  s3_bucket = "sce.sentinel2"  # Update the AWS bucket
  s3_input_folder = "2021_input_dir" # update the AWS bucket input folder containing the downloaded the sentinel images
  drive_input_dir = SCE_folder + "input_dir/" 
  drive_output_dir = SCE_folder + "output_dir/" 
  s3_output_folder = "2021_output_dir" # update the AWS bucket folder containing the downloaded the sentinel images
  session = boto3.Session()
  client_config = botocore.config.Config(max_pool_connections=50)
  s3_client = session.client('s3',**get_aws_creds(), config=client_config)
  try:
    s3_output_contents = s3_client.list_objects_v2(Bucket = s3_bucket,Prefix=s3_output_folder)["Contents"]
    s3_output_files = [content['Key'].split("/")[1].replace(" ", "") for content in s3_output_contents]
  except:
    s3_output_files = []

  image_file_names = get_s3_files(s3_client, s3_bucket, s3_input_folder)

  group_files = grouping_files(image_file_names)

  error_files = []

  print("Number of Files to be processed:",len(group_files))

  def preprocess_images(files):
    band_file_regex_pattern= r"^\d{2}_[A-Za-z]_[A-Za-z]{2}_\d{4}_\d{1,2}_\d{1,2}_\d{1,2}_B(?!01|09|10)\d{1,2}\.jp2$"
    band_files = [item for item in files if re.match(band_file_regex_pattern, item)]
    detfoo_files = [i for i in files if "DETFOO" in i]
    qualit_files = [i for i in files if "QUALIT" in i]
    output_file = "_".join(band_files[0].split("_")[0:6]) + ".tif"
    # if(output_file in s3_output_files or output_file in error_files or len(band_files)!=9 or ("_6_5_" not in output_file and "_6_18" not in output_file)):
    if(output_file in s3_output_files or output_file in error_files or len(band_files)!=9 or "KU_2021" not in output_file):
      return
    # delete_all_files_from_trash()


    band_files_path = [drive_input_dir + i for i in band_files]
    deftoo_files_path = [drive_input_dir + i for i in detfoo_files]
    qualit_files_path = [drive_input_dir + i for i in qualit_files]
    full_output_file = ""

    try:

      for index in range(len(band_files)):
        download_from_s3(s3_client, s3_bucket, s3_input_folder+"/"+band_files[index], band_files_path[index])

      for index in range(len(detfoo_files)):
        download_from_s3(s3_client, s3_bucket, s3_input_folder+"/"+detfoo_files[index], deftoo_files_path[index])

      output_file = "_".join(band_files[0].split("_")[0:6]) + ".tif"

      print("Processing: ", output_file)

      full_output_file = drive_output_dir + output_file


      resample_bands(drive_input_dir)
      mask_clouds_and_stack_bands(band_files_path, deftoo_files_path, full_output_file)
      upload_file(full_output_file,s3_bucket, s3_output_folder+'/ {}'.format(output_file))

    except Exception as e:
      print(e)
      error_files.append(output_file)

    files_to_be_deleted = band_files_path + deftoo_files_path + qualit_files_path
    files_to_be_deleted = band_files_path + deftoo_files_path + qualit_files_path
    files_to_be_deleted.append(full_output_file)
    delete_files(files_to_be_deleted)

  for files in reversed(group_files):
    if (len(files)==0 or files[0]==""):
      continue
    preprocess_images(files)
